In [147]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

Importing our numeric data

In [148]:
df = pd.read_csv('data/players_with features_no_nan_only_num_values.csv')

In [149]:
df.head(5)

,height,weight,games,victories,ties,defeats,goals,yellowCards,yellowReds,redCards,skin_colour,gravity,meanIAT_yellowCards,meanIAT_yellowReds,meanIAT_redCards,meanIAT_gravity,meanExp_yellowCards,meanExp_yellowReds,meanExp_redCards,meanExp_gravity
0,177.0,72.0,144.0,65.0,32.0,47.0,10.0,21.0,1.0,2.0,0.375,116.674213,0.058858,0.000535,0.001203,0.114196,0.093395,0.001087,0.002447,0.205938
1,179.0,82.0,427.0,183.0,101.0,143.0,88.0,33.0,0.0,2.0,0.750,95.589050,0.020451,0.000000,0.001752,0.075289,0.013638,0.000000,-0.000520,-0.002640
2,182.0,71.0,654.0,247.0,179.0,228.0,9.0,19.0,0.0,0.0,0.125,19.000000,0.011768,0.000000,0.000000,0.011768,0.018965,0.000000,0.000000,0.018965
3,187.0,80.0,285.0,138.0,57.0,90.0,28.0,50.0,4.0,3.0,0.125,276.224227,0.054788,0.006257,0.008396,0.524554,0.085066,0.011986,0.018192,1.050962
4,180.0,68.0,214.0,88.0,54.0,72.0,11.0,34.0,0.0,0.0,1.000,34.000000,0.049788,0.000000,0.000000,0.049788,0.077372,0.000000,0.000000,0.077372


Creating target data:

In [150]:
target = df['skin_colour']

In [151]:
target.unique()

array([ 0.375,  0.75 ,  0.125,  1.   ,  0.25 ,  0.   ,  0.5  ,  0.875,
        0.625])

Start by binary classification

In [152]:
def separator(x):
    if x == 0.:
        return 0
    elif x == 0.125:
        return 1
    elif x == 0.25:
        return 2
    elif x == 0.375:
        return 3
    elif x == 0.5:
        return 4
    elif x == 0.625:
        return 5
    elif x == 0.75:
        return 6
    elif x == 0.875:
        return 7
    elif x == 1.:
        return 8

In [153]:
target = target.apply(separator)

In [154]:
target.head()

0    3
1    6
2    1
3    1
4    8
Name: skin_colour, dtype: int64

In [155]:
cols = ['height', 'weight', 'games', 'victories', 'ties', 'defeats', 'goals', 'skin_colour']
df = df.drop(cols, axis = 1)
data = df.as_matrix()

In [156]:
data.shape

(1406, 12)

In [157]:
target.shape

(1406,)

In [158]:
target.unique

<bound method IndexOpsMixin.unique of 0       3
1       6
2       1
3       1
4       8
5       2
6       0
7       0
8       4
9       0
10      0
11      0
12      4
13      0
14      0
15      1
16      1
17      0
18      0
19      0
20      1
21      2
22      1
23      1
24      0
25      1
26      0
27      0
28      0
29      0
       ..
1376    0
1377    2
1378    2
1379    0
1380    2
1381    0
1382    2
1383    2
1384    2
1385    2
1386    4
1387    1
1388    0
1389    1
1390    0
1391    4
1392    2
1393    4
1394    1
1395    6
1396    2
1397    0
1398    0
1399    7
1400    4
1401    0
1402    8
1403    6
1404    7
1405    2
Name: skin_colour, dtype: int64>

In [117]:
data1 = data[:1000]
target1 = target[:1000]

In [138]:
forest = RandomForestClassifier(n_estimators = 100)

In [69]:
data2 = data[1000:]
target2 = target[1000:]

In [139]:
forest.fit(data, target)

ValueError: Unknown label type: array([[ 0.375],
       [ 0.75 ],
       [ 0.125],
       ..., 
       [ 0.75 ],
       [ 0.875],
       [ 0.25 ]])

In [70]:
test = forest.predict(data2)

In [72]:
truth = [test == target2]

In [78]:
len(truth[0])

406

In [79]:
truth[0].value_counts()

True     301
False    105
Name: skin_colour, dtype: int64

In [80]:
forest.feature_importances_

array([ 0.09700738,  0.0238374 ,  0.02386248,  0.10753348,  0.13245097,
        0.06335541,  0.05887908,  0.11584132,  0.13810492,  0.05693551,
        0.05840193,  0.12379011])

In [81]:
i = _

In [82]:
i

array([ 0.09700738,  0.0238374 ,  0.02386248,  0.10753348,  0.13245097,
        0.06335541,  0.05887908,  0.11584132,  0.13810492,  0.05693551,
        0.05840193,  0.12379011])

In [83]:
sum(i)

0.99999999999999989

In [94]:
df.columns

Index(['yellowCards', 'yellowReds', 'redCards', 'gravity',
       'meanIAT_yellowCards', 'meanIAT_yellowReds', 'meanIAT_redCards',
       'meanIAT_gravity', 'meanExp_yellowCards', 'meanExp_yellowReds',
       'meanExp_redCards', 'meanExp_gravity'],
      dtype='object')

In [108]:
f.feature_importances_

NotFittedError: Estimator not fitted, call `fit` before `feature_importances_`.

In [159]:
from sklearn.cross_validation import cross_val_score

In [160]:
f = RandomForestClassifier(n_estimators = 100)

In [128]:
target = target.values

In [144]:
target = np.array(target)

In [161]:
scores = cross_val_score(f, data, target, cv=10, scoring='accuracy')

In [162]:
scores

array([ 0.26896552,  0.26896552,  0.22916667,  0.26760563,  0.21428571,
        0.27857143,  0.26618705,  0.22627737,  0.25547445,  0.23357664])

idea: use a hold-out sample